**CLASIFICACIÓN PARTIENDO DE IMÁGENES**

En este notebook vamos a tratar de clasificar los apartamentos según su precio en baratos, medios o caros. Para ello estableceremos los límites en 50€ para los baratos y 150€ para los caros.
Dicha predicción se va a hacer a partir de las imágenes que tenemos en el dataset de airbnb que venimos usando en las prácticas de este Bootcamp.

En primer lugar nos descargamos el fichero de internet y lo copiamos en un directorio local de My Drive donde tenemos recogido todo el entorno de esta práctica.
A continuación hacemos lo mismo con las imágenes de cada una de las entradas. Usamos la vista en miniatura que sacamos de la URL de dicho fichero.

También montamos el google collab con My Drive para tenerlo vinculado.

Estos pasos solo hay que realizarlos la primera vez, una vez que tenemos los ficheros en My Drive se pueden saltar y pasamos a cargar los datos directamente desde dicho directorio.

In [ ]:
# nos descargamos el dataset de OpenDataSoft
!wget -O "airbnb-listings.csv" "https://public.opendatasoft.com/explore/dataset/airbnb-listings/download/?format=csv&disjunctive.host_verifications=true&disjunctive.amenities=true&disjunctive.features=true&refine.country=Spain&q=Madrid&timezone=Europe/London&use_labels_for_header=true&csv_separator=%3B"

!ls -lah

--2020-06-25 06:01:03--  https://public.opendatasoft.com/explore/dataset/airbnb-listings/download/?format=csv&disjunctive.host_verifications=true&disjunctive.amenities=true&disjunctive.features=true&refine.country=Spain&q=Madrid&timezone=Europe/London&use_labels_for_header=true&csv_separator=%3B
Resolving public.opendatasoft.com (public.opendatasoft.com)... 34.249.199.226, 34.248.20.69
Connecting to public.opendatasoft.com (public.opendatasoft.com)|34.249.199.226|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/csv]
Saving to: ‘airbnb-listings.csv’

airbnb-listings.csv     [  <=>               ]  54.19M  2.77MB/s    in 50s     

2020-06-25 06:02:01 (1.09 MB/s) - ‘airbnb-listings.csv’ saved [56826824]

total 55M
drwxr-xr-x 1 root root 4.0K Jun 25 06:01 .
drwxr-xr-x 1 root root 4.0K Jun 25 05:59 ..
-rw-r--r-- 1 root root  55M Jun 25 06:02 airbnb-listings.csv
drwxr-xr-x 1 root root 4.0K Jun 19 16:15 .config
drwx------ 4 root root 4.0K Jun 

In [ ]:
!ls -lah

total 16K
drwxr-xr-x 1 root root 4.0K Jun 17 16:18 .
drwxr-xr-x 1 root root 4.0K Jun 26 05:15 ..
drwxr-xr-x 1 root root 4.0K Jun 19 16:15 .config
drwxr-xr-x 1 root root 4.0K Jun 17 16:18 sample_data


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!cp airbnb-listings.csv "drive/My Drive/BootcampBD&ML/práctica/prácticaDeepLearning"

In [ ]:
# aquí creamos nuestra estructura de datos, que va a consistir en la url de la
# imagen y un índice para saber donde insertarla en nuestro array
images_paths = [[i, img_url] for i, img_url in enumerate(full_df['Thumbnail Url'])]
images_paths[:5]

[[0,
  'https://a0.muscache.com/im/pictures/cffe393a-0d84-4fd5-ab4c-a62e067c1b0d.jpg?aki_policy=small'],
 [1,
  'https://a0.muscache.com/im/pictures/ea919e56-aa99-4d5d-a129-1edf0d117d6a.jpg?aki_policy=small'],
 [2,
  'https://a0.muscache.com/im/pictures/57011236/eea5c213_original.jpg?aki_policy=small'],
 [3,
  'https://a0.muscache.com/im/pictures/974f0245-55c2-4e8c-b9bf-14c1c975c798.jpg?aki_policy=small'],
 [4,
  'https://a0.muscache.com/im/pictures/c2dde263-20dd-43af-8c6b-be636c2c0ce1.jpg?aki_policy=small']]

In [ ]:
import imageio as io
import cv2

# esta es la función que se descargará la imagen y devolverá la imagen y el 
# índice indicando la posición donde se incrustará la imagen en nuestro array
def get_image(data_url, target_size=(224, 224)):
    idx, url = data_url
    try:
        img = io.imread(url)
        # hay alguna imagen en blanco y negro y daría error al incluirla en 
        # nuestro array de imagenes que tiene 3 canales, así que convertimos
        # todas las imágenes que tengan menos de 3 dimensiones a color
        if img.ndim < 3:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        img = cv2.resize(img, dsize=target_size)
        return img, idx
    except IOError as err:
        return (None, idx)

In [ ]:
import numpy as np

# en este array iremos incrustando las imágenes conforme las vayamos obteniendo
loaded_images = np.zeros((len(images_paths), 224, 224, 3), dtype=np.uint8)

# y en este array llevaremos un control de cuales se han cargado correctamente
# y cuales no
was_loaded = np.zeros(len(images_paths))

In [ ]:
import concurrent
from tqdm import tqdm

# creamos un pool de procesos que se irán descargando las imágenes
# por defecto, se crearán tantos como CPUs tenga vuestra máquina
with concurrent.futures.ProcessPoolExecutor() as executor:
    # procesamos la lista de urls de imágenes paralelizandola con el pool de procesos
    for (img, idx) in tqdm(executor.map(get_image, images_paths), total=len(images_paths)):
        # metemos la imagen en nuestro array
        if img is not None:
            loaded_images[idx] = img
            was_loaded[idx] = 1
        else:
            was_loaded[idx] = 0

print('Terminado!')
print(f'Total de imágenes recuperadas correctamente: {sum(was_loaded)}/{len(images_paths)}')

100%|██████████| 14001/14001 [08:14<00:00, 28.29it/s]

Terminado!
Total de imágenes recuperadas correctamente: 11271.0/14001


In [ ]:
# guardamos las imágenes (y yo os recomiendo que os lo guardéis en GDrive para evitar tener que repetir esto)
np.save('images.npy', loaded_images)
np.save('was_loaded.npy', was_loaded)

In [ ]:
# almacenamos las imagenes en nuestro drive
!cp images.npy "drive/My Drive/BootcampBD&ML/práctica/prácticaDeepLearning"
!cp was_loaded.npy "drive/My Drive/BootcampBD&ML/práctica/prácticaDeepLearning"

Esta parte de descarga, montado y copiado solo hace falta ejecutarla la primera vez. Una vez que lo tenemos copiado solo necesitamos cargarlo directamente.

A partir de aquí empieza nuestro ejercicio de clasificación.

Como hábito de buena costumbre, para no incurrir en errores involuntarios, en primer lugar se va a dividir el dataset original en train, validation y test.

Se trabaja únicamente con el de train con el objetivo de elegir un modelo. Eso se verifica con el conjunto de validation y finalmente se aplica ese "entrenamiento" al bloque de test.

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline
cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


#hacemos la divisón en train, val y test
full_df = pd.read_csv('drive/My Drive/BootcampBD&ML/práctica/prácticaDeepLearning/airbnb-listings.csv', sep=';', decimal='.')
full_train, test = train_test_split(full_df, test_size=0.2, shuffle=True, random_state=0)
train, val = train_test_split(full_train, test_size=0.2, shuffle=True, random_state=0)

print(f'Dimensiones del dataset de training: {train.shape}')
print(f'Dimensiones del dataset de validación: {val.shape}')
print(f'Dimensiones del dataset de test: {test.shape}')

# Guardamos
train.to_csv('./train.csv', sep=';', decimal='.', index=True)
val.to_csv('./val.csv', sep=';', decimal='.', index=True)
test.to_csv('./test.csv', sep=';', decimal='.', index=True)


Dimensiones del dataset de training: (8960, 89)
Dimensiones del dataset de validación: (2240, 89)
Dimensiones del dataset de test: (2801, 89)


In [ ]:
#cargamos las imágenes desde el directorio de My Drive (ya las habíamos descargado previamente)
images  = np.load('drive/My Drive/BootcampBD&ML/práctica/prácticaDeepLearning/images.npy')
was_loaded  = np.load('drive/My Drive/BootcampBD&ML/práctica/prácticaDeepLearning/was_loaded.npy')

#cargamos los datos ya divididos en train, val y test
df_train = pd.read_csv('./train.csv', sep=';', decimal='.')
df_val = pd.read_csv('./val.csv', sep=';', decimal='.')
df_test = pd.read_csv('./test.csv', sep=';', decimal='.')

#usando el índice de la división anterior obtenemos los conjuntos de test, val y test en las imágenes
train_imgs = images[df_train['Unnamed: 0']]
val_imgs = images[df_val['Unnamed: 0']]
test_imgs = images[df_test['Unnamed: 0']]

train_was_loaded = was_loaded[df_train['Unnamed: 0']]
val_was_loaded = was_loaded[df_val['Unnamed: 0']]
test_was_loaded = was_loaded[df_test['Unnamed: 0']]

print(f'Dimensiones del dataset de training: {train_imgs.shape}')
print(f'Dimensiones del dataset de validación: {val_imgs.shape}')
print(f'Dimensiones del dataset de test: {test_imgs.shape}')

print(f'Dimensiones del dataset de training: {train_was_loaded.shape}')
print(f'Dimensiones del dataset de validación: {val_was_loaded.shape}')
print(f'Dimensiones del dataset de test: {test_was_loaded.shape}')


In [ ]:
# nos quedamos con los datos e imágenes para los que hemos podido encontrar imágenes
train_imgs_loaded = train_imgs[train_was_loaded == 1]
val_imgs_loaded = val_imgs[val_was_loaded == 1]
test_imgs_loaded = test_imgs[test_was_loaded == 1]

train_with_imgs = df_train[train_was_loaded == 1]
val_with_imgs = df_val[val_was_loaded == 1]
test_with_imgs = df_test[test_was_loaded == 1]

print(f'Dimensiones del dataset de training: {train_imgs_loaded.shape}')
print(f'Dimensiones del dataset de validación: {val_imgs_loaded.shape}')
print(f'Dimensiones del dataset de test: {test_imgs_loaded.shape}')

print(f'Dimensiones del dataset de training: {train_with_imgs.shape}')
print(f'Dimensiones del dataset de validación: {val_with_imgs.shape}')
print(f'Dimensiones del dataset de test: {test_with_imgs.shape}')

Dimensiones del dataset de training: (7204, 224, 224, 3)
Dimensiones del dataset de validación: (1790, 224, 224, 3)
Dimensiones del dataset de test: (2277, 224, 224, 3)
Dimensiones del dataset de training: (7204, 90)
Dimensiones del dataset de validación: (1790, 90)
Dimensiones del dataset de test: (2277, 90)


In [ ]:
#PRICE
#imputamos valores vacíos con la media de train
MeanPriceTrain = train_with_imgs['Price'].mean()
train_with_imgs['Price'].fillna(MeanPriceTrain, inplace=True)
val_with_imgs['Price'].fillna(MeanPriceTrain, inplace=True)
test_with_imgs['Price'].fillna(MeanPriceTrain, inplace=True)
#definimos outlier >400€
#indexTrainFiltered = train_with_imgs[train_with_imgs['Price']>400].index
#train_with_imgs.drop(indexTrainFiltered, inplace=True)
#indexValFiltered = val_with_imgs[val_with_imgs['Price']>400].index
#val_with_imgs.drop(indexValFiltered, inplace=True)
#indexTestFiltered = test_with_imgs[test_with_imgs['Price']>400].index
#test_with_imgs.drop(indexTestFiltered, inplace=True)

#train_imgs_loaded.drop(indexTrainFiltered, inplace=True)
#val_imgs_loaded.drop(indexValFiltered, inplace=True)
#test_imgs_loaded.drop(indexTestFiltered, inplace=True)

#transformamos variable Price a gausiana
train_with_imgs['Price'] = train_with_imgs['Price'].apply(lambda x: np.log10(x))
val_with_imgs['Price'] = val_with_imgs['Price'].apply(lambda x: np.log10(x))
test_with_imgs['Price'] = test_with_imgs['Price'].apply(lambda x: np.log10(x))
#categorizamos la variable precio en 3 tipos: barato (0), medio (1) y caro (2).
train_with_imgs['Cat_Price'] = train_with_imgs['Price'].apply(lambda x: 0 if x < np.log10(50) else (1 if x < np.log10(150) else 2))
val_with_imgs['Cat_Price'] = val_with_imgs['Price'].apply(lambda x: 0 if x < np.log10(50) else (1 if x < np.log10(150) else 2))
test_with_imgs['Cat_Price'] = test_with_imgs['Price'].apply(lambda x: 0 if x < np.log10(50) else (1 if x < np.log10(150) else 2))

print(f'Dimensiones del dataset de training: {train_imgs_loaded.shape}')
print(f'Dimensiones del dataset de validación: {val_imgs_loaded.shape}')
print(f'Dimensiones del dataset de test: {test_imgs_loaded.shape}')

print(f'Dimensiones del dataset de training: {train_with_imgs.shape}')
print(f'Dimensiones del dataset de validación: {val_with_imgs.shape}')
print(f'Dimensiones del dataset de test: {test_with_imgs.shape}')

Dimensiones del dataset de training: (7204, 224, 224, 3)
Dimensiones del dataset de validación: (1790, 224, 224, 3)
Dimensiones del dataset de test: (2277, 224, 224, 3)
Dimensiones del dataset de training: (7204, 91)
Dimensiones del dataset de validación: (1790, 91)
Dimensiones del dataset de test: (2277, 91)


In [ ]:
#Redimensionamos las imágenes de entrada. Estoy teniendo problemas de RAM y no puedo ejecutarlo
#con 224x224 no puedo escalar /255. Con 112x112 no puedo ejecutar el modelo
#es necesario asumir esta pérdida de información
train_imgs_loaded = np.resize(train_imgs_loaded, (train_imgs_loaded.shape[0],64, 64, train_imgs_loaded.shape[3]))
val_imgs_loaded = np.resize(val_imgs_loaded, (val_imgs_loaded.shape[0],64, 64, val_imgs_loaded.shape[3]))
test_imgs_loaded = np.resize(test_imgs_loaded, (test_imgs_loaded.shape[0],64, 64, test_imgs_loaded.shape[3]))

print(f'Dimensiones del dataset de training: {train_imgs_loaded.shape}')
print(f'Dimensiones del dataset de training: {val_imgs_loaded.shape}')
print(f'Dimensiones del dataset de training: {test_imgs_loaded.shape}')

Dimensiones del dataset de training: (7204, 64, 64, 3)
Dimensiones del dataset de training: (1790, 64, 64, 3)
Dimensiones del dataset de training: (2277, 64, 64, 3)


In [ ]:
#escalamos los datos de entrada. Lo hago en celdas separadas ya que hay algún problema de RAM
#se trata de imágenes así que no hace falta centrar, solo dividimos por el máximo. 
# nos aseguramos de hacerlo como float para no perder la info de los decimales

train_imgs_loaded = train_imgs_loaded.astype('float32') / 255.


In [ ]:
val_imgs_loaded = val_imgs_loaded.astype('float32') / 255.


In [ ]:
test_imgs_loaded = test_imgs_loaded.astype('float32') / 255.

In [ ]:
from keras.utils import to_categorical

# convertimos las etiquetas a one-hot encoding
num_classes = 3
Ytrain = to_categorical(train_with_imgs['Cat_Price'], num_classes)
Yval = to_categorical(val_with_imgs['Cat_Price'], num_classes)
Ytest = to_categorical(test_with_imgs['Cat_Price'], num_classes)

Using TensorFlow backend.


Una vez que ya tenemos los datos de las imágenes y la variable objetivo preparados, definimos los modelos con los que vamos a ir trabajando.

La idea general es usar una red neuronal convolucional (CNN) donde las primeras capas actúan como extractor de características y añadimos un clasificador final.

In [ ]:
# imports necesarios
import numpy as np
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers.core import Dense, Flatten
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
from keras.layers.pooling import MaxPooling2D
from keras.layers import BatchNormalization, Activation
from keras.layers import Dropout
from keras.utils import to_categorical

# Inizializamos el modelo
model = Sequential()

# Definimos una capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), input_shape=(64, 64, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Definimos una segunda capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Definimos una tercera capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Añadimos nuestro clasificador
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compilamos el modelo
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001, decay=1e-6),
              metrics=['accuracy'])

# Entrenamos el modelo
model.fit(train_imgs_loaded, Ytrain,
          batch_size=128,
          shuffle=True,
          epochs=10,
          validation_data=(val_imgs_loaded, Yval)) 


Train on 7204 samples, validate on 1790 samples
Epoch 1/10
7204/7204 [==============================] - 10s 1ms/step - loss: 1.6351 - accuracy: 0.4554 - val_loss: 1.0973 - val_accuracy: 0.4480
Epoch 2/10
7204/7204 [==============================] - 10s 1ms/step - loss: 1.0859 - accuracy: 0.4622 - val_loss: 1.0961 - val_accuracy: 0.4480
Epoch 3/10
7204/7204 [==============================] - 10s 1ms/step - loss: 0.9507 - accuracy: 0.4729 - val_loss: 1.0945 - val_accuracy: 0.4480
Epoch 4/10
7204/7204 [==============================] - 10s 1ms/step - loss: 0.9193 - accuracy: 0.4804 - val_loss: 1.0931 - val_accuracy: 0.4480
Epoch 5/10
7204/7204 [==============================] - 10s 1ms/step - loss: 0.9129 - accuracy: 0.4738 - val_loss: 1.0919 - val_accuracy: 0.4480
Epoch 6/10
7204/7204 [==============================] - 10s 1ms/step - loss: 0.9045 - accuracy: 0.4736 - val_loss: 1.0894 - val_accuracy: 0.4480
Epoch 7/10
7204/7204 [==============================] - 10s 1ms/step - loss: 0.901

En los tres bloques convolucionales que estamos definiendo se observa la misma estructura: una capa convolucional, una capa de batchNormalization, otra de MaxPooling y otra de Dropout. Estas tres últimas tienen como objetivo diezmar el contenido de la salida de la capa convolucional. Esto se hace para quitar complejidad y reducir el overfitting del modelo. En los siguientes apartados iremos probando con distintos valores y configuraciones a ver cuál es el mejor resultado que podemos obtener.

In [ ]:
#quito el BatchNormalization

# Inizializamos el modelo
model = Sequential()

# Definimos una capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Definimos una segunda capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Definimos una tercera capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Añadimos nuestro clasificador
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compilamos el modelo
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001, decay=1e-6),
              metrics=['accuracy'])

# Entrenamos el modelo
model.fit(train_imgs_loaded, Ytrain,
          batch_size=128,
          shuffle=True,
          epochs=10,
          validation_data=(val_imgs_loaded, Yval)) 

Train on 7204 samples, validate on 1790 samples
Epoch 1/10
7204/7204 [==============================] - 7s 1ms/step - loss: 0.9146 - accuracy: 0.4618 - val_loss: 0.9430 - val_accuracy: 0.4955
Epoch 2/10
7204/7204 [==============================] - 7s 907us/step - loss: 0.8954 - accuracy: 0.4713 - val_loss: 0.9592 - val_accuracy: 0.4944
Epoch 3/10
7204/7204 [==============================] - 7s 919us/step - loss: 0.8975 - accuracy: 0.4636 - val_loss: 0.9637 - val_accuracy: 0.4978
Epoch 4/10
7204/7204 [==============================] - 7s 910us/step - loss: 0.8942 - accuracy: 0.4672 - val_loss: 0.9543 - val_accuracy: 0.4721
Epoch 5/10
7204/7204 [==============================] - 7s 911us/step - loss: 0.8922 - accuracy: 0.4771 - val_loss: 0.9727 - val_accuracy: 0.4520
Epoch 6/10
7204/7204 [==============================] - 7s 916us/step - loss: 0.8928 - accuracy: 0.4772 - val_loss: 0.9597 - val_accuracy: 0.4950
Epoch 7/10
7204/7204 [==============================] - 7s 913us/step - loss: 

In [ ]:
#ahora añadimos MaxNormalization en el clasificador

from keras.constraints import max_norm

# Inizializamos el modelo
model = Sequential()

# Definimos una capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Definimos una segunda capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Definimos una tercera capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Añadimos nuestro clasificador
model.add(Flatten())
model.add(Dense(1024, activation='relu', kernel_constraint=max_norm(3.)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compilamos el modelo
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001, decay=1e-6),
              metrics=['accuracy'])

# Entrenamos el modelo
model.fit(train_imgs_loaded, Ytrain,
          batch_size=128,
          shuffle=True,
          epochs=10,
          validation_data=(val_imgs_loaded, Yval)) 

Train on 7204 samples, validate on 1790 samples
Epoch 1/10
7204/7204 [==============================] - 7s 989us/step - loss: 0.9146 - accuracy: 0.4771 - val_loss: 0.9747 - val_accuracy: 0.4950
Epoch 2/10
7204/7204 [==============================] - 7s 927us/step - loss: 0.8988 - accuracy: 0.4678 - val_loss: 0.9836 - val_accuracy: 0.4827
Epoch 3/10
7204/7204 [==============================] - 7s 931us/step - loss: 0.8973 - accuracy: 0.4653 - val_loss: 0.9835 - val_accuracy: 0.4665
Epoch 4/10
7204/7204 [==============================] - 7s 921us/step - loss: 0.8952 - accuracy: 0.4678 - val_loss: 0.9762 - val_accuracy: 0.4464
Epoch 5/10
7204/7204 [==============================] - 7s 927us/step - loss: 0.8929 - accuracy: 0.4646 - val_loss: 0.9766 - val_accuracy: 0.4916
Epoch 6/10
7204/7204 [==============================] - 7s 931us/step - loss: 0.8949 - accuracy: 0.4557 - val_loss: 0.9860 - val_accuracy: 0.4944
Epoch 7/10
7204/7204 [==============================] - 7s 933us/step - loss

Viendo los tres resultados anteriores son muy similares, pero parece que este último es algo mejor 49% de accuracy frente a 47% y 48%).

Por esta razón seguimos con este modelo y aumentamos el número de épocas.

In [ ]:
# Inizializamos el modelo
model = Sequential()

# Definimos una capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Definimos una segunda capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Definimos una tercera capa convolucional
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Añadimos nuestro clasificador
model.add(Flatten())
model.add(Dense(1024, activation='relu', kernel_constraint=max_norm(3.)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compilamos el modelo
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001, decay=1e-6),
              metrics=['accuracy'])

# Entrenamos el modelo
model.fit(train_imgs_loaded, Ytrain,
          batch_size=128,
          shuffle=True,
          epochs=50,
          validation_data=(val_imgs_loaded, Yval)) 

Train on 7204 samples, validate on 1790 samples
Epoch 1/50
7204/7204 [==============================] - 7s 991us/step - loss: 0.9112 - accuracy: 0.4732 - val_loss: 0.9528 - val_accuracy: 0.4480
Epoch 2/50
7204/7204 [==============================] - 7s 925us/step - loss: 0.9006 - accuracy: 0.4679 - val_loss: 0.9851 - val_accuracy: 0.4989
Epoch 3/50
7204/7204 [==============================] - 7s 922us/step - loss: 0.8959 - accuracy: 0.4704 - val_loss: 0.9623 - val_accuracy: 0.4950
Epoch 4/50
7204/7204 [==============================] - 7s 918us/step - loss: 0.8945 - accuracy: 0.4697 - val_loss: 0.9660 - val_accuracy: 0.4955
Epoch 5/50
7204/7204 [==============================] - 7s 925us/step - loss: 0.8914 - accuracy: 0.4808 - val_loss: 0.9639 - val_accuracy: 0.4939
Epoch 6/50
7204/7204 [==============================] - 7s 922us/step - loss: 0.8928 - accuracy: 0.4695 - val_loss: 0.9596 - val_accuracy: 0.4978
Epoch 7/50
7204/7204 [==============================] - 7s 928us/step - loss

Vemos que aunque aumentemos el número de épocas nuestro modelo se comporta prácticamente igual.

Llegados a este punto en el que estamos haciendo bastantes pruebas de forma manual, vamos a usar Hyper-opt para optimizar nuestros resultados y encontrar las características del modelo que mejor prestaciones nos de.

In [ ]:
# instalamos los paquetes necesarios
!pip install networkx==1.11 # para instala hyperopt correctamente, si no, da errores
!pip install hyperopt

     |████████████████████████████████| 1.3MB 3.5MB/s 
ERROR: scikit-image 0.16.2 has requirement networkx>=2.0, but you'll have networkx 1.11 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: networkx 2.4
    Uninstalling networkx-2.4:
      Successfully uninstalled networkx-2.4


Vamos a trabajar con 4 variables:

- el número de filtros en las capas convolucionales
- el porcentaje de dropout
- el número de neuronas en la capa densa
- el tamaño del kernel en las capas convolucionales

In [ ]:
# imports necesarios
import sys
import time
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from keras.callbacks import EarlyStopping


# definimos nuestro espacio de búsqueda
# vamos a variar:
# - el número de filtros en nuestras capas conv
# - el porcentaje de dropout
# - el número de neuronas en la capa dense
# - el tamaño del kernel en las capas conv
space = {
    'n_filters_conv': hp.choice('n_filters_conv', [32, 64, 128]),
    'dropout': hp.uniform('dropout', 0.0, 0.5),
    'neurons_dense': hp.choice('neurons_dense', [256, 512, 1024]), 
    'kernel_size' : hp.choice('kernel_size', [3, 5])
}

def	get_callbacks(pars):
  callbacks	= [EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2, verbose=0, mode='auto')]
  return callbacks

def mi_cnn(pars):
  print ('Parameters: ', pars)
  # Inizializamos el modelo
  model = Sequential()

  # Definimos una capa convolucional
  model.add(Conv2D(pars['n_filters_conv'], kernel_size=(pars['kernel_size']), input_shape=(64, 64, 3)))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(pars['dropout']))

  # Definimos una segunda capa convolucional
  model.add(Conv2D(pars['n_filters_conv'], kernel_size=(pars['kernel_size']), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(pars['dropout']))

  # Definimos una tercera capa convolucional
  model.add(Conv2D(pars['n_filters_conv'], kernel_size=(pars['kernel_size']), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(pars['dropout']))

  # Añadimos nuestro clasificador
  model.add(Flatten())
  model.add(Dense(pars['neurons_dense'], activation='relu', kernel_constraint=max_norm(3.)))
  model.add(Dropout(pars['dropout']))
  model.add(Dense(num_classes, activation='softmax'))

  # Compilamos el modelo
  model.compile(loss='categorical_crossentropy',
                optimizer=Adam(lr=0.0001, decay=1e-6),
                metrics=['accuracy'])

  # Entrenamos el modelo
  history = model.fit(train_imgs_loaded, Ytrain,
            batch_size=128,
            shuffle=True,
            epochs=50,
            validation_data=(val_imgs_loaded, Yval),
            verbose = 0,
            callbacks = get_callbacks(pars)) 
  

  best_epoch_loss = np.argmin(history.history['val_loss'])
  best_val_loss = np.min(history.history['val_loss'])
  best_val_acc = np.max(history.history['val_accuracy'])
  
  print('Epoch {} - val acc: {} - val loss: {}'.format(best_epoch_loss, best_val_acc, best_val_loss))
  sys.stdout.flush()
  
  return {'loss': best_val_loss, 'best_epoch': best_epoch_loss, 'eval_time': time.time(), 'status': STATUS_OK, 'model': model, 'history': history}


trials = Trials()
best = fmin(mi_cnn, space, algo=tpe.suggest, max_evals=10, trials=trials)
print(best)

Parameters: 
{'dropout': 0.007405141224077039, 'kernel_size': 5, 'n_filters_conv': 64, 'neurons_dense': 512}
Epoch 2 - val acc: 0.4949720799922943 - val loss: 0.8708138468545242
Parameters: 
{'dropout': 0.15950979926671216, 'kernel_size': 3, 'n_filters_conv': 128, 'neurons_dense': 512}
Epoch 0 - val acc: 0.4949720799922943 - val loss: 0.8738649612698475
Parameters: 
{'dropout': 0.2185707044649131, 'kernel_size': 5, 'n_filters_conv': 32, 'neurons_dense': 512}
Epoch 3 - val acc: 0.4949720799922943 - val loss: 0.8767173019867369
Parameters: 
{'dropout': 0.0030657473607762498, 'kernel_size': 3, 'n_filters_conv': 128, 'neurons_dense': 1024}
Epoch 1 - val acc: 0.4949720799922943 - val loss: 0.8731627184585486
Parameters: 
{'dropout': 0.12805414951968475, 'kernel_size': 5, 'n_filters_conv': 128, 'neurons_dense': 1024}
Epoch 3 - val acc: 0.4949720799922943 - val loss: 0.8743143003080144
Parameters: 
{'dropout': 0.3913452147594918, 'kernel_size': 3, 'n_filters_conv': 32, 'neurons_dense': 1024}


Viendo los resultados no superamos el 49% de accuracy en validación, así que vamos a tomarlo como definitivo y pasamos a evaluar nuestro modelo final.

{'dropout': 0.007405141224077039, 'kernel_size': 1, 'n_filters_conv': 1, 'neurons_dense': 1}

Parameters: 
{'dropout': 0.007405141224077039, 'kernel_size': 5, 'n_filters_conv': 64, 'neurons_dense': 512}
Epoch 2 - val acc: 0.4949720799922943 - val loss: 0.8708138468545242

64 filtros, 512 neuronas y 5x5 el kernel.

In [ ]:
# Inizializamos el modelo definitivo con los parámetros optimizados
modelDef = Sequential()

# Definimos una capa convolucional
modelDef.add(Conv2D(64, kernel_size=(5, 5), input_shape=(64, 64, 3)))
modelDef.add(MaxPooling2D(pool_size=(2, 2)))
modelDef.add(Dropout(0.00745))

# Definimos una segunda capa convolucional
modelDef.add(Conv2D(64, kernel_size=(5, 5), activation='relu'))
modelDef.add(MaxPooling2D(pool_size=(2, 2)))
modelDef.add(Dropout(0.00745))

# Definimos una tercera capa convolucional
modelDef.add(Conv2D(64, kernel_size=(5, 5), activation='relu'))
modelDef.add(MaxPooling2D(pool_size=(2, 2)))
modelDef.add(Dropout(0.00745))

# Añadimos nuestro clasificador
modelDef.add(Flatten())
modelDef.add(Dense(1024, activation='relu', kernel_constraint=max_norm(3.)))
modelDef.add(Dropout(0.0074))
modelDef.add(Dense(num_classes, activation='softmax'))

# Compilamos el modelo
modelDef.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001, decay=1e-6),
              metrics=['accuracy'])

# Entrenamos el modelo
modelDef.fit(train_imgs_loaded, Ytrain,
          batch_size=128,
          shuffle=True,
          epochs=10,
          validation_data=(val_imgs_loaded, Yval)) 

Train on 7204 samples, validate on 1790 samples
Epoch 1/10
7204/7204 [==============================] - 5s 662us/step - loss: 0.9584 - accuracy: 0.4647 - val_loss: 0.8761 - val_accuracy: 0.4950
Epoch 2/10
7204/7204 [==============================] - 4s 496us/step - loss: 0.8868 - accuracy: 0.4643 - val_loss: 0.8743 - val_accuracy: 0.4480
Epoch 3/10
7204/7204 [==============================] - 4s 494us/step - loss: 0.8870 - accuracy: 0.4663 - val_loss: 0.8752 - val_accuracy: 0.4480
Epoch 4/10
7204/7204 [==============================] - 4s 499us/step - loss: 0.8864 - accuracy: 0.4686 - val_loss: 0.8713 - val_accuracy: 0.4950
Epoch 5/10
7204/7204 [==============================] - 4s 494us/step - loss: 0.8864 - accuracy: 0.4646 - val_loss: 0.8712 - val_accuracy: 0.4950
Epoch 6/10
7204/7204 [==============================] - 4s 494us/step - loss: 0.8859 - accuracy: 0.4629 - val_loss: 0.8716 - val_accuracy: 0.4950
Epoch 7/10
7204/7204 [==============================] - 4s 497us/step - loss

Por último evaluamos el modelo con el conjunto de test.

In [ ]:
# Evaluamos el modelo
scores = modelDef.evaluate(test_imgs_loaded, Ytest)

print('Loss: %.3f' % scores[0])
print('Accuracy: %.3f' % scores[1])


2277/2277 [==============================] - 1s 359us/step
Loss: 0.895
Accuracy: 0.477


**CONCLUSIÓN**

No me quedo muy conforme con el resultado del 47% o 49% de accuracy. Se puede deber a la baja calidad de las imágenes (importante la limitación de haber reducido tanto el tamaño), a la dificultad del dataset y al desbalanceo del mismo. Aunque no descartaría algún error mío en el procesamiento. Por más que lo he revisado no he encontrado nada.

Viendo las predicciones (abajo está copiado un ejemplo) se observa como siempre queda casi al 50% entre los pisos baratos y medios, y esto podría tener su explicación en que se trata de un dataset muy desbalanceado donde hay pocos pisos caros.



array([[0.47080284, 0.46632615, 0.06287099],
       [0.47080284, 0.46632615, 0.06287099],
       [0.47080284, 0.46632615, 0.06287099],
       [0.47080284, 0.46632615, 0.06287099],
       [0.47080284, 0.46632615, 0.06287099],